# (DRAFT) Define LSP metrics across Aus and determine causal drivers

load NDVI data --> smooth --> remove non-seasonal regions --> summarise time-series over ecoregions --> calculate phenometrics --> load climate data --> summarise climate data over ecoregions --> select/summarise climate data into sensible metrics --> develop statistical relationships between phenometrics & climate (e.g. causal networks, partial correlations etc.)

***

**To Do:**
* Split cropping class into rain-fed and irrigated classes (can be split it into cereal and pasture as well?)
* ~~Remove "Trobriand Islands rain forests" and "Sumba deciduous forests", "Timor and Wetar deciduous forests" (in Indonesia) from eco-regions~~
* ~~Multiprocess the climate variable extraction as its currently very slow running sequentially~~
* Currently using defaults from phenolopy for extracting phenometrics - consider how we might best extract metrics (switching to R might be better for this). At least inspect the results for each ecoregion and assess phenolopy's ability to extract sensible phenology.
* Use literature to determine best way to extract climate variables corresponding with phenology metrics - in this draft workflow I only extract mean climate over the preceeding two months of a given metric.
* Consider sourcing climate data from somewhere other than ANUClim, perhaps ERA5

In [ ]:
# %load_ext autoreload
# %autoreload 2
%matplotlib inline
import os
import sys
import math
import warnings
import xarray as xr
import rioxarray as rxr
import pandas as pd
import geopandas as gpd
import seaborn as sb
import xarray as xr
import numpy as np
import scipy.signal
import contextily as ctx
import matplotlib.pyplot as plt
from datetime import datetime
from odc.geo.xr import assign_crs
from odc.geo.geom import Geometry
import multiprocessing as mp
from tqdm import tqdm
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

# from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import shap
from scipy import stats

sys.path.append('/g/data/os22/chad_tmp/Aus_phenology/src')
# from xr_phenology import xr_phenology
from phenolopy import calc_phenometrics

sys.path.append('/g/data/os22/chad_tmp/dea-notebooks/Tools/')
from dea_tools.spatial import xr_vectorize
from dea_tools.classification import HiddenPrints

# sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
# from _utils import start_local_dask
# from _percentile import xr_quantile

## Analysis Parameters


In [ ]:
var='NDVI'
ds_path = '/g/data/os22/chad_tmp/AusENDVI/results/publication/AusENDVI-clim_MCD43A4_gapfilled_1982_2022.nc'
chunks=None#dict(latitude=1000, longitude=1000, time=-1)
base = '/g/data/os22/chad_tmp/Aus_phenology/'

## Open data

In [ ]:
ds = assign_crs(xr.open_dataset(ds_path, chunks=chunks), crs='EPSG:4326')
ds = ds.rename({'AusENDVI_clim_MCD43A4':'NDVI'})
ds = ds['NDVI']

## Smoothing filters

In [ ]:
ds_up = ds.resample(time="2W").interpolate("linear")

# # Savitsky-Golay
ds_smooth = xr.apply_ufunc(
        scipy.signal.savgol_filter,
        ds_up,
        input_core_dims=[['time']],
        output_core_dims=[['time']],
        kwargs=dict(
            window_length=11,
            polyorder=3,
            deriv=0,
            mode='interp'),
        dask='allowed'
    )
#SG cuts of last 6 months so clip to complete calendar years.
ds_smooth = ds_smooth.sel(time=slice('1982', '2021'))

#smoothing reorders dims for some reason
ds_smooth = ds_smooth.transpose('time', 'latitude','longitude')

In [ ]:
fig,ax=plt.subplots(1,1, figsize=(12,4))
ds.mean(['latitude', 'longitude']).plot(ax=ax, label='NDVI')
ds_smooth.mean(['latitude', 'longitude']).plot(ax=ax, label='NDVI Savitsky-Golay')
ax.legend(loc='upper left')
ax.set_title('Signal smoothing of NDVI');

## Create a mask for non-seasonal regions

To avoid applying phenology trend analysis on regions that do not experience regular seasonal variation, we will create a mask that removes areas designated as 'non-seasonal'. We use [Moore et al. 2016.](https://bg.copernicus.org/articles/13/5085/2016/) as a guide here ("non-seasonally dynamic": EVI varies but not in accordance with seasons), the thresholds needed to create the mask will be somewhat arbitrary but we will attempt to delineate areas with 1. low seasonal variability and low NDVI, and 2. areas with low seasonal variability and high interannual variability.

Process:
1. Calculate seasonally-adjusted anomalies
2. Calcuate standard deviation in anomalies
3. Overall mean NDVI
4. Standard deviation of the mean seasonal pattern.


In [ ]:
# import warnings
# warnings.simplefilter('ignore')

# #standardized anom
# def anomalies(ds):
#     return xr.apply_ufunc(
#         lambda x, m: (x - m),
#             ds.groupby("time.month"),
#             ds.groupby("time.month").mean()
#     )

# # 3. Mean NDVI
# mean_ndvi = ds_smooth.mean('time')

# ##Mask very low NDVI (get rid of remnant water etc.)
# low_ndvi_mask = xr.where(mean_ndvi<0.115,0, 1)
# mean_ndvi = mean_ndvi.where(low_ndvi_mask)
    
# # 1. calculate anomalies
# ndvi_std_anom = anomalies(ds_smooth)
# # 2. std dev in anomalies
# anom_std = ndvi_std_anom.std('time').where(low_ndvi_mask)
# # 4. Std deviation of the mean seasonal pattern
# std_mean_season = ds.groupby("time.month").mean().std('month').where(low_ndvi_mask)

### Create the mask

In [ ]:
# low_seasonal_variability_low_ndvi = xr.where((std_mean_season<0.03) & (mean_ndvi<0.3), 1, 0)
# low_seasonal_variability_high_anomaly = xr.where((std_mean_season<0.03) &  (anom_std>0.07), 1, 0)

# seasonal_mask = xr.where((low_seasonal_variability_low_ndvi) | (low_seasonal_variability_high_anomaly), 0, 1)

# #aply the mask
# ds_smooth = ds_smooth.where(seasonal_mask)

## Ecoregions & cropping regions

From here: https://ecoregions.appspot.com/

Ecoregions is natural ecologies only, we need to include a cropping class which we get from GFSAD

> To Do: split cropping class into rain-fed and irrigated classes
> Remove "Trobriand Islands rain forests" and "Sumba deciduous forests" (in Indonesia)

In [ ]:
gdf = gpd.read_file('/g/data/os22/chad_tmp/Aus_phenology/data/Ecoregions2017_aus_processed.geojson')

In [ ]:
# # #open ecoregions
# gdf = gpd.read_file('/g/data/os22/chad_tmp/Aus_phenology/data/Ecoregions2017_aus.geojson')

# # #clip to seasonal_mask
# # mask_gdf = xr_vectorize(~seasonal_mask.astype('bool'), dtype='int16')
# # mask_gdf = mask_gdf[mask_gdf['attribute']==0.0]
# # mask_gdf['zone'] = 1
# # mask_gdf = mask_gdf.dissolve(by='zone', aggfunc='sum')
# # gdf = gpd.clip(gdf, mask_gdf)

# #create a cropping geometery
# crop = assign_crs(xr.open_dataarray('/g/data/os22/chad_tmp/NEE_modelling/data/1km/Landcover_1km_monthly_2002_2021.nc'), crs='epsg:4326').isel(time=1)
# crop = (crop==40)
# crop = xr_vectorize(crop, dtype='int16')
# crop = crop[crop['attribute']==1.0]
# crop['zone'] = 1
# crop = crop.dissolve(by='zone', aggfunc='sum')

# #remove areas in ecoregions where crops are
# gdf = gpd.overlay(gdf, crop, how='symmetric_difference')

# #add cropping polygons as a new geomtry to ecoregions
# crop['ECO_NAME'] = 'Cropping' #add same attrs as eco
# crop['BIOME_NAME'] = 'Cropping'
# crop['BIOME_NUM'] = '0.0'
# crop = crop.drop('attribute',axis=1).reset_index(drop=True)
# gdf = pd.concat([gdf, crop]) #join
# gdf = gdf.drop('attribute',axis=1).dropna(axis=0, how='any').reset_index(drop=True) #tidy
# #remove tiny polys in Indo.
# gdf = gdf[~gdf['ECO_NAME'].isin(["Trobriand Islands rain forests",
#                                  "Sumba deciduous forests",
#                                  "Timor and Wetar deciduous forests",
#                                  "Louisiade Archipelago rain forests",
#                                  "New Guinea mangroves",
#                                 "Southeast Papuan rain forests",
#                                 "Timor and Wetar deciduous forests"])].reset_index(drop=True)

# gdf.to_file('/g/data/os22/chad_tmp/Aus_phenology/data/Ecoregions2017_aus_processed.geojson')

In [ ]:
a = gdf.plot(column='ECO_NAME', figsize=(8,8), legend=False, cmap='tab20b');
ctx.add_basemap(a, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
plt.title('Ecoregions', fontsize=14);

## Summarise NDVI & extract phenometrics over ecoregions

In [ ]:
results={}
i=0
for index, row in gdf.iterrows():
    print("Feature {:02}/{:02}\r".format(i + 1, len(range(0, len(gdf)))), end="")
    
    #clip to ecoregion
    geom = Geometry(geom=row.geometry, crs=gdf.crs)
    xx = ds_smooth.odc.mask(poly=geom)
    
    #summarise into 1d timeseries
    xx = xx.mean(['latitude', 'longitude'])

    #calculate phenometrics
    
    with HiddenPrints():
        warnings.simplefilter('ignore')
        phen = xx.groupby('time.year').map(calc_phenometrics)

    #add to dict
    results[row.ECO_NAME] = phen
    
    i+=1

## Test extraction of phenometrics

## Climate data etc.

load climate data --> summarise climate data over ecoregions --> select/summarise climate data into sensible metrics

Consider getting climate from ERA5 instead.

In [ ]:
base_clim = '/g/data/os22/chad_tmp/AusENDVI/data/5km/'
co2 = xr.open_dataset(base_clim+'CO2_5km_monthly_1982_2022.nc')
rain = xr.open_dataset(base_clim+'rain_5km_monthly_1981_2022.nc').sel(time=slice('1982','2022')).drop_vars('spatial_ref')
srad = xr.open_dataset(base_clim+'srad_5km_monthly_1982_2022.nc').drop_vars('spatial_ref')
tavg = xr.open_dataset(base_clim+'tavg_5km_monthly_1982_2022.nc').drop_vars('spatial_ref')
vpd = xr.open_dataset(base_clim+'vpd_5km_monthly_1982_2022.nc').drop_vars('spatial_ref')
# rain_cml3 = xr.open_dataset(base+'rain_cml3_5km_monthly_1982_2022.nc').drop_vars('spatial_ref')

climate = xr.merge([co2, rain, srad, tavg, vpd])
climate = assign_crs(climate, crs='EPSG:4326')
climate = climate.transpose('time', 'latitude','longitude')

climate = climate.sel(time=slice('1982', '2021')) #match trimmed NDVI

### Summarise climate data over polygons

This is multiprocessed for faster processing. Enter `ncpus`

Note, this will use way more memory than if run sequentially

In [ ]:
ncpus=10

In [ ]:
def clim_timeseries(row, ds, results_dict):
    # clip to ecoregion
    geom = Geometry(geom=row.geometry, crs=gdf.crs)
    yy = ds.odc.mask(poly=geom)
    
    #summarise into 1d timeseries
    yy = yy.mean(['latitude', 'longitude'])

    results_dict[row.ECO_NAME] = yy

# parallel function for above function
def _parallel_fun(df, ds, ncpus):

    manager = mp.Manager()
    results_dict = manager.dict()

    # progress bar
    pbar = tqdm(total=len(gdf))

    def update(*a):
        pbar.update()

    with mp.Pool(ncpus) as pool:
        for index, row in df.iterrows():
            pool.apply_async(
                clim_timeseries,
                [row, ds, results_dict],
                callback=update,
            )
        
        pool.close()
        pool.join()
        pbar.close()

    return results_dict

# run the parallel function
results_clim = _parallel_fun(gdf, climate, ncpus=ncpus)
results_clim = results_clim._getvalue() #bring into memory

# run the sequential function
# results_dict={}
# for index, row in gdf.iterrows():
#     results_clim = clim_timeseries(row, climate, results_dict)

### Calculate climate metrics that relate to phenometrics

Let's start with peak of season (POS) as thats an easy one.

For each ecoregion, find the average date of POS, then we can calculate annual summary statistics around they POS date.  Let's do the three-months preceeding the POS, inclusive. So if the peak is in June, then we can calculate summary statistics from April-May-June...that'll be a good start

In [ ]:
def months_filter(month, start, end):
    return (month >= start) & (month <= end)

metrics_to_extract = ['pos'] #'sos', 'eos', 'los'

for metric in metrics_to_extract:
    print(metric)

    if not os.path.exists(f'{base}/data/{metric}'):
        os.makedirs(f'{base}/data/{metric}')
    
    results_pheno_clim={}
    # i=0
    for index, row in gdf.iterrows():
        # print("Feature {:02}/{:02}\r".format(i + 1, len(range(0, len(gdf)))), end="")
    
        #open corresponding phenometrics
        pheno = results[row.ECO_NAME]
    
        #if all Nans then skip
        if np.sum(np.isnan(pheno[f'{metric}_values'].values)) == len(pheno.year):
            print('skip', row.ECO_NAME)
            continue
            
        #find average time for POS - just use a random year
        # doys = [int(i) for i in pheno['pos_times'].values]
        # years = [i for i in pheno.year.values]
        # times = [pd.Timestamp(datetime.strptime(f'{y} {d}', '%Y %j')) for y, d in zip(years,doys)]
        mean_pos = pd.Timestamp(datetime.strptime(f'{2000} {int(pheno.pos_times.mean().values.item())}', '%Y %j'))
    
        #subtract 2 months to find the month-range for summarising climate
        months_before = mean_pos - pd.DateOffset(months=1)
    
        ##NEED TO IMPROVE THIS SO WHEN MONTHS GO FROM 12-2 FOR EXAMPLE IT STILL WORKS
        months_range = list(range(months_before.month, mean_pos.month+1, 1))
        # print(months_range)
    
        #now we meed to index the climate data by the range of months
        trimmed_climate = results_clim[row.ECO_NAME].sel(time=months_filter(results_clim[row.ECO_NAME]['time.month'],
                                                                            months_range[0],months_range[-1]))
    
        #calculate annual climate summary stats (just mean for now)
        rain=trimmed_climate['rain']
        trimmed_climate = trimmed_climate[['CO2', 'srad', 'tavg', 'vpd']]
        annual_climate = trimmed_climate.groupby('time.year').mean()
        rain=rain.groupby('time.year').sum()
        annual_climate['rain'] = rain
    
        #join our POS metric to the climate data
        annual_climate[f'{metric}_values'] = pheno[f'{metric}_values']
        if metric != 'los':
            annual_climate[f'{metric}_times'] = pheno[f'{metric}_times']
    
        #add to results dict
        results_pheno_clim[row.ECO_NAME] = annual_climate

    # #export model input data
    for k,v in results_pheno_clim.items():
        n=k.replace(" ", "_")
        results_pheno_clim[k].drop('spatial_ref').to_dataframe().to_csv(f'{base}data/{metric}/{n}_{metric}_model_data.csv')
        

## Develop basic model to predict change in phenometrics

The goal here is to determine the most important features using shapely, this is one methods we might get to 'causality'

Start with just a single phenometric and single location

In [ ]:
path = '/g/data/os22/chad_tmp/Aus_phenology/data/pos/Jarrah-Karri_forest_and_shrublands_pos_model_data.csv'
var= 'pos_values'

### Exploratory Plot

In [ ]:
df = pd.read_csv(path, index_col='year')
df = df.rolling(1, min_periods=1).mean()
df = df.dropna()
df = df[['pos_values', 'pos_times', 'CO2', 'rain', 'srad', 'tavg', 'vpd']]
df.plot(subplots=True, layout=(2,4), figsize=(14,6));

### Create a ML model

And use Shap feature importance to determine most important features

In [ ]:
# x = df[['CO2','rain', 'tavg', 'vpd', 'srad']]
# y = df[var]

# rf = RandomForestRegressor(n_estimators=300, random_state = 1).fit(x, y)
# prediction = rf.predict(x)
# # mse = mean_squared_error(y, prediction)
# # rmse = mse**.5
# # r2 = r2_score(y, prediction)
# # print(r2)

# # Lets plot the predictions versus the real values
# plt.plot(y.values, label='Real')
# plt.plot(prediction)


In [ ]:
### Feature importance using SHAP

In [ ]:
# explainer = shap.Explainer(rf)
# shap_values = explainer(x)

# # fig, ax = plt.subplots(1,1, figsize=(5,7))
# shap.plots.bar(shap_values, max_display=10, show=True)
# # ax = plt.gca() 

### Iterative linear modelling

To determine the impact each variable has on predicting the slope of phenometric

In [ ]:
#first fit a model with all vars
x = df[['CO2','rain', 'tavg', 'vpd', 'srad']]
y = df[var]

lr = LinearRegression().fit(x, y)
prediction = lr.predict(x)
r2_all = r2_score(y, prediction)

#calculate slope of predicted variable with all params
s_actual, i, r, p, se = stats.linregress(df.index, y)
s, i, r, p, se = stats.linregress(df.index, prediction)
print(s_actual, s)

fig,ax=plt.subplots(1,1, figsize=(7,4))
ax.plot(y.values, label='Observed')
ax.plot(prediction, label='All')

# now fit a model without a given variable
# and calculate the slope of the phenometric
r_delta={}
s_delta={}
for v in ['CO2','rain', 'tavg', 'vpd', 'srad']:
    #set variable as constant 
    constant = x[v].mean()
    xx = x.drop(v, axis=1)
    xx[v] = constant

    #model and determine slope
    lr = LinearRegression().fit(xx, y)
    pred = lr.predict(xx)
    r2 = r2_score(y, pred)
    s_p, i, r, p, se = stats.linregress(df.index, pred)
    
    # plt.plot(y.values, label='Real')
    ax.plot(pred, label=v)
    #determine the eucliden distance between
    #modelled slope and actual (and r2)
    s_delta[v] = math.dist((s,), (s_p,))
    r_delta[v] = math.dist((r2_all,), (r2,))

ax.legend()
ax.set_xticks(ticks = range(0,40, 4), labels=range(1982,2022, 4))
ax.set_ylabel('vPOS (NDVI)')
ax.set_title('Jarrah-Karri Forest')

s_delta = pd.Series(s_delta)
r_delta = pd.Series(r_delta)
sensivity = pd.concat([s_delta, r_delta], axis=1).rename({0:'Slope_difference', 1:'r2_difference'}, axis=1)
sensivity

## Loop through all regions and extract key variable

Using iterative linear modelling

In [ ]:
base = '/g/data/os22/chad_tmp/Aus_phenology/'
metrics_to_extract = ['pos'] #'sos', 'eos', 'los'
var = 'pos_values'

In [ ]:
gdf = gpd.read_file('/g/data/os22/chad_tmp/Aus_phenology/data/Ecoregions2017_aus_processed.geojson')

In [ ]:
for metric in metrics_to_extract:
    files = [f for f in os.listdir(f'{base}data/{metric}/') if f.endswith('.csv') ]
    files.sort()
    print(metric)
    
    dffs = []
    for f in files:
        name = f.removesuffix('_'+metric+'_model_data.csv').replace("_", " ")
        # print('', name)
        
        #open data
        df = pd.read_csv(f'{base}data/{metric}/{f}', index_col='year')
        df = df[[metric+'_values', metric+'_times', 'CO2', 'rain', 'srad', 'tavg', 'vpd']]
        df = df.dropna()
        
        #first fit a model with all vars
        x = df[['CO2','rain', 'tavg', 'vpd', 'srad']]
        y = df[var]
        
        lr = LinearRegression().fit(x, y)
        prediction = lr.predict(x)
        #calculate slope of predicted variable with all params
        s_actual, i, r, p_actual, se = stats.linregress(df.index, y)
        s_prediction, i, r, p, se = stats.linregress(df.index, prediction)
        # print('  ', s_actual, s_prediction)
        r2_all = r2_score(y, prediction)
        
        # now fit a model without a given variable
        # and calculate the slope of the phenometric
        r_delta={}
        s_delta={}
        for v in ['CO2','rain', 'tavg', 'vpd', 'srad']:
            #set variable of interest as a constant value 
            constant = x[v].mean()
            xx = x.drop(v, axis=1)
            xx[v] = constant
        
            #model and determine slope
            lrr = LinearRegression().fit(xx, y)
            pred = lrr.predict(xx)
            r2 = r2_score(y, pred)
            s_p, i, r, p, se = stats.linregress(df.index, pred)

            #determine the eucliden distance between
            #modelled slope and actual slope (and r2)
            s_delta[v] = math.dist((s_actual,), (s_p,))
            r_delta[v] = math.dist((r2_all,), (r2,))

        #determine most important feature
        s_delta = pd.Series(s_delta)
        r_delta = pd.Series(r_delta)
        fi = pd.concat([s_delta, r_delta], axis=1).rename({0:'slope_difference', 1:'r2_difference'}, axis=1)
        fi = fi.loc[[fi['slope_difference'].idxmax()]]
        fi = fi.reset_index().rename({'index':'feature'},axis=1)

        #create tidy df
        fi['ECO_NAME'] = name
        fi['phenometric'] = var
        fi['slope_actual'] = s_actual
        fi['slope_modelled'] = s_prediction
        fi['p_values'] = p_actual
        fi['r2'] = r2_all
        dffs.append(fi)
    
    top_features = pd.concat(dffs).reset_index(drop=True)
    gdf_with_feature = gdf.merge(top_features, on='ECO_NAME')
      

### Plot

In [ ]:
fig,ax=plt.subplots(1,3, figsize=(18,5), sharey=True, layout='constrained')

significant = gdf_with_feature[gdf_with_feature['p_values'] <= 0.1]

gdf_with_feature.plot(edgecolor="black", linewidth=0.05,facecolor='none', ax=ax[0])
a=significant.plot(column='slope_actual', ax=ax[0], legend=True, cmap='BrBG', vmax=0.002,
                   vmin=-0.002, legend_kwds={'shrink':0.8}) #  edgecolor="black", linewidth=0.1 cmap='BrBG',
ctx.add_basemap(a, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)

gdf_with_feature.plot(edgecolor="black", linewidth=0.05,facecolor='none', ax=ax[1])
cmap = ['tab:blue','tab:green','tab:orange','tab:red','tab:purple']
a=significant.plot(column='feature', ax=ax[1], legend=False, cmap=ListedColormap(cmap)) # 
ctx.add_basemap(a, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)

ax[1].legend(
        [Patch(facecolor=cmap[0]), Patch(facecolor=cmap[1]), Patch(facecolor=cmap[2]),Patch(facecolor=cmap[3]), Patch(facecolor=cmap[4])], 
        ['CO2','rain', 'tavg', 'vpd', 'srad'],
         loc = 'upper right'
    );

gdf_with_feature.plot(edgecolor="black", linewidth=0.05,facecolor='none', ax=ax[2])
a=significant.plot(column='slope_difference', ax=ax[2], legend=True, vmin=0, vmax=0.0014, cmap='magma', legend_kwds={'shrink':0.8})
ctx.add_basemap(a, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)

ax[0].set_title('vPOS Slope (NDVI/yr, p<=0.1) 1982-2021')
ax[1].set_title('vPOS most important variable');
ax[2].set_title('Sensitivity ('+u'Δslope'+')');

In [ ]:
fig,ax=plt.subplots(1,3, figsize=(18,5), sharey=True, layout='constrained')

significant = gdf_with_feature[gdf_with_feature['p_values'] <= 0.1]


gdf_with_feature.plot(edgecolor="black", linewidth=0.05,facecolor='none', ax=ax[0])
a=significant.plot(column='slope_actual', ax=ax[0], legend=True, cmap='coolwarm', vmax=1.5,
                   vmin=-1.5, legend_kwds={'shrink':0.8}) #  edgecolor="black", linewidth=0.1 cmap='BrBG',
ctx.add_basemap(a, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)

gdf_with_feature.plot(edgecolor="black", linewidth=0.05,facecolor='none', ax=ax[1])
cmap = ['tab:blue','tab:green','tab:orange','tab:red','tab:purple']
a=significant.plot(column='feature', ax=ax[1], legend=False, cmap=ListedColormap(cmap)) # 
ctx.add_basemap(a, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)

ax[1].legend(
        [Patch(facecolor=cmap[0]), Patch(facecolor=cmap[1]), Patch(facecolor=cmap[2]),Patch(facecolor=cmap[3]), Patch(facecolor=cmap[4])], 
        ['CO2','rain', 'tavg', 'vpd', 'srad'],
         loc = 'upper right'
    );

gdf_with_feature.plot(edgecolor="black", linewidth=0.05,facecolor='none', ax=ax[2])
a=significant.plot(column='slope_difference', ax=ax[2], legend=True,cmap='magma', vmin=0, vmax=1.5, legend_kwds={'shrink':0.8})
ctx.add_basemap(a, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)

ax[0].set_title('POS Slope (days/yr, p<=0.1) 1982-2021')
ax[1].set_title('POS most important variable');
ax[2].set_title('Sensitivity ('+u'Δslope' + ')');

## Test causal networks

In [ ]:
import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests.parcorr import ParCorr

In [ ]:
df = df[['pos_values','CO2', 'rain', 'tavg', 'vpd','srad']]
data = df.values
T, N = data.shape
var_names = list(df.columns)

# Initialize dataframe object, specify time axis and variable names
dataframe = pp.DataFrame(data, 
                         datatime = {0:np.arange(len(data))}, 
                         var_names=var_names)

# tp.plot_timeseries(dataframe, figsize=(15, 5));

parcorr = ParCorr(significance='analytic')
pcmci = PCMCI(
    dataframe=dataframe, 
    cond_ind_test=parcorr,
    verbosity=1)

correlations = pcmci.run_bivci(tau_max=1, val_only=True)['val_matrix']
matrix_lags = np.argmax(np.abs(correlations), axis=2)

# tp.plot_densityplots(dataframe=dataframe, setup_args={'figsize':(15, 10)}, add_densityplot_args={'matrix_lags':matrix_lags}); plt.show()

tau_max = 0
pc_alpha = 0.01
pcmci.verbosity = 0

results = pcmci.run_pcmciplus(tau_min=0, tau_max=tau_max, pc_alpha=pc_alpha)

In [ ]:
# print("MCI partial correlations")
# print(results['val_matrix'].round(2))

In [ ]:
# tp.plot_graph(
#     figsize=(8,4),
#     val_matrix=results['val_matrix'],
#     graph=results['graph'],
#     var_names=var_names,
#     link_colorbar_label='cross-MCI (edges)',
#     node_colorbar_label='auto-MCI (nodes)',
#     ); plt.show()